# new model

torch 모델을 다른것을 사용해서 확인하기에, 새로운 그리드 파일을 연것.

encoder, transformer 이미지를 테스트 예정.

encoder 이미지는 1개라서 그리드는 필요없음.

In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import math



p = os.path.abspath('../') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# data 분류

In [3]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('D:/project/voice_pathology_ai/voice_data/all_data/pathology/phrase/*.wav')
healthy = glob('D:/project/voice_pathology_ai/voice_data/all_data/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1354:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  1354
Healthy:  634
총 데이터수 :  1988
---
훈련 셋 :  1590 Counter({'pathology': 1083, 'healthy': 507})
테스트 셋 :  398 Counter({'pathology': 271, 'healthy': 127})
---


In [4]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 



In [5]:
#2. trainset에만 random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res




 fold0 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold1 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold2 
before dataset shape Counter({'pathology': 866, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 866, 'healthy': 866})

 fold3 
before dataset shape Counter({'pathology': 867, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 867, 'healthy': 867})

 fold4 
before dataset shape Counter({'pathology': 867, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 867, 'healthy': 867})


In [6]:
import pickle

 
#load
with open("../../voice_data/all_data/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

# data set

In [7]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

In [8]:
# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes

        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

# data loader

In [9]:
#3. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [10]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.

#확인을 위해 데이터셋 하나만 확인



train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_train_list,
                                                   classes,
                                               data_num=0,
                                               training=True
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.

validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               data_num=0,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 



In [11]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# model

https://pytorch.org/audio/main/tutorials/speech_recognition_pipeline_tutorial.html


dat1=torch.randn((32,12,318,768))
model1=nn.Conv2d(12,3,(3,3))
dat2=model1(dat1)
print(dat2.size())
#torch.Size([32, 3, 316, 766])
res_18_sample = models.resnet18(pretrained=True)
dat3=res_18_sample(dat2)
print(dat3.size())



----

In [13]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [12]:

#mean 변경 및 데이터 변경 실험
        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.sum(axis=1)/(hidden_feature!=0).sum(axis=1)### 수정 zero 포함않게.
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [27]:
###
# model for enocoder

sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(512,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(256,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        x = encoder_x
        #x = self.zero_padding(x,encoder_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [ ]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.


bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model_wav2vec = bundle.get_model().to(DEVICE)

print(model_wav2vec.__class__)

        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        self.model_wav2vec = bundle.get_model().to(DEVICE)
        self.res_18 = models.resnet18(pretrained=True)
        self.conv_model1=nn.Conv2d(12,3,(3,3))
        
        #input_dim =101951
        
        self.fc = nn.Sequential(nn.Linear(1000,512),
                                nn.BatchNorm1d(512),
                                nn.ReLU(),
                                nn.Linear(512,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Linear(256,50),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=[ feat.mean(axis=1) for feat  in  hidden_feature]
        return hidden_feature
        
        
    def forward(self,x):
        with torch.inference_mode():
            x,_ = self.model_wav2vec.extract_features(x)
        x = torch.stack(x,axis=1)
        x = self.conv_model1(x)
        x = self.res_18(x)
        x= self.fc(x)
    
        return x 
    


In [15]:
#7. Optimizer, Objective Function
def model_initialize(num):
    model = wav_path_Net(num).to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [14]:
#7. Optimizer, Objective Function
def model_initialize():
    model = wav_path_Net().to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [20]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label,sig_len) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        sig_len = sig_len.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image,sig_len) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy


In [21]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            
            #true.false값을 sum해줌. item
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy


In [22]:
#데이터 로더 제작 함수

def load_data(data_ind):

    train_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_train_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=True
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,
                                               ) # 순서가 암기되는것을 막기위해.

    validation_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_valid_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=False
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 
    return train_loader,validation_loader



In [23]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def test_evaluate(model,test_loader):
    model.eval()
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            output = F.softmax(output, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
        return predictions,answers

In [24]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    
    
    for grid_num in range(0,24):
        
        average_accuracy = 0
        average_fscore = 0
        for data_ind in range(1,6):
            model=model_initialize(grid_num)
            check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(grid_num)+'-'+str(data_ind)+'-alldata-phrase.pt'
            model.load_state_dict(torch.load(check_path))

            predictions,answers = test_evaluate(model, test_loader)
            predictions=[ dat.cpu().numpy() for dat in predictions]
            answers=[ dat.cpu().numpy() for dat in answers]


            cf = confusion_matrix(answers, predictions)
            cf_list.append(cf)

            acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
            average_accuracy+=acc
            precision=cf[0,0]/(cf[0,0]+cf[1,0])
            recall=cf[0,0]/(cf[0,0]+cf[0,1])
            #fscore=2*precision*recall/(precision+recall)

            #fscroe macro추가
            fscore = f1_score(answers,predictions,average='macro')
            average_fscore+=fscore
            

            print('{}번 모델 / {}번 fold'.format(grid_num+1,data_ind))
            print("Accuracy : {:.4f}% ".format(acc*100))
            print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
            print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
            print("f score : {:.4f} ".format(fscore))
            print(cf)
            print("-----")
        print("----")
        print("* {}번 모델 {:.4f}% / score: {:.4f}".format(grid_num+1, 100*average_accuracy/5,average_fscore/5))
        print("----")


In [31]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    for data_ind in range(1,2):
        model=model_initialize(grid_num)
        check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'-alldata-phrase.pt'
        model.load_state_dict(torch.load(check_path))

        predictions,answers = test_evaluate(model, test_loader)
        predictions=[ dat.cpu().numpy() for dat in predictions]
        answers=[ dat.cpu().numpy() for dat in answers]


        cf = confusion_matrix(answers, predictions)
        cf_list.append(cf)

        acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
        precision=cf[0,0]/(cf[0,0]+cf[1,0])
        recall=cf[0,0]/(cf[0,0]+cf[0,1])
        #fscore=2*precision*recall/(precision+recall)

        #fscroe macro추가
        fscore = f1_score(answers,predictions,average='macro')

        print('{}번 모델'.format(grid_num+1))
        print("Accuracy : {:.4f}% ".format(acc*100))
        print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
        print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
        print("f score : {:.4f} ".format(fscore))
        print(cf)
        print("-----")




In [28]:
#10. 학습 및 평가.
# kfold 적용
train_accs = []
valid_accs = []
for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'-alldata-phrase.pt'
    print(check_path)
    early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
    train_loader,validation_loader = load_data(data_ind-1)

    best_train_acc=0 # accuracy 기록용
    best_valid_acc=0

    model=model_initialize()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


    print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
    for Epoch in range(1,EPOCHS+1):
        train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
        valid_loss,valid_accuracy = evaluate(model, validation_loader)


        print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
              format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))


        early_stopping(valid_loss, model)
        if -early_stopping.best_score == valid_loss:
            best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

        if early_stopping.early_stop:
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
                print("[{} 교차검증] Early stopping".format(data_ind))
                break

        if Epoch==EPOCHS:
            #만약 early stop 없이 40 epoch라서 중지 된 경우.
            train_accs.append(best_train_acc)
            valid_accs.append(best_valid_acc)

../checkpoint/checkpoint_w2v_large_model60k_encoder-1.pt


TypeError: __init__() missing 1 required positional argument: 'num'

In [25]:
#10. 학습 및 평가.
# kfold 적용



all_train_accs =[]
all_valid_accs =[]


for num in range(24):
    print("{}-번 레이어".format(num))
    train_accs = []
    valid_accs = []

    for data_ind in range(1,6):

        check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(num)+'-'+str(data_ind)+'-alldata-phrase.pt'
        print(check_path)
        early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
        train_loader,validation_loader = load_data(data_ind-1)

        best_train_acc=0 # accuracy 기록용
        best_valid_acc=0

        model=model_initialize(num)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


        print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
        for Epoch in range(1,EPOCHS+1):
            train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
            valid_loss,valid_accuracy = evaluate(model, validation_loader)


            print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
                  format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
            

            early_stopping(valid_loss, model)
            if -early_stopping.best_score == valid_loss:
                best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

            if early_stopping.early_stop:
                    train_accs.append(best_train_acc)
                    valid_accs.append(best_valid_acc)
                    print("[{} 교차검증] Early stopping".format(data_ind))
                    break

            if Epoch==EPOCHS:
                #만약 early stop 없이 40 epoch라서 중지 된 경우.
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
    all_train_accs.append(train_accs)
    all_valid_accs.append(valid_accs)

0-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_0-1-alldata-phrase.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0163	 Train Acc:74.71 %  | 	Valid Loss:0.0138 	Valid Acc: 81.45 %

Validation loss decreased (inf --> 0.013843).  Saving model ...

[EPOCH:2]	 Train Loss:0.0142	 Train Acc:81.47 %  | 	Valid Loss:0.0131 	Valid Acc: 81.13 %

Validation loss decreased (0.013843 --> 0.013133).  Saving model ...

[EPOCH:3]	 Train Loss:0.0121	 Train Acc:83.37 %  | 	Valid Loss:0.0129 	Valid Acc: 81.45 %

Validation loss decreased (0.013133 --> 0.012898).  Saving model ...

[EPOCH:4]	 Train Loss:0.0125	 Train Acc:83.78 %  | 	Valid Loss:0.0128 	Valid Acc: 81.76 %

Validation loss decreased (0.012898 --> 0.012757).  Saving model ...

[EPOCH:5]	 Train Loss:0.0114	 Train Acc:85.80 %  | 	Valid Loss:0.0120 	Valid Acc: 82.39 %

Validation loss decreased (0.012757 --> 0.012019).  Saving model ...

[EPOCH:6]	 Train Loss:0.0105	 Train Acc:86.20 %  | 	Valid Loss:0.0127 	Valid Acc: 78.62 %

EarlyStoppi


[EPOCH:10]	 Train Loss:0.0103	 Train Acc:85.01 %  | 	Valid Loss:0.0115 	Valid Acc: 85.53 %

EarlyStopping counter: 2 out of 5

[EPOCH:11]	 Train Loss:0.0096	 Train Acc:86.45 %  | 	Valid Loss:0.0116 	Valid Acc: 85.85 %

EarlyStopping counter: 3 out of 5

[EPOCH:12]	 Train Loss:0.0098	 Train Acc:86.45 %  | 	Valid Loss:0.0120 	Valid Acc: 84.59 %

EarlyStopping counter: 4 out of 5

[EPOCH:13]	 Train Loss:0.0088	 Train Acc:87.72 %  | 	Valid Loss:0.0120 	Valid Acc: 85.22 %

EarlyStopping counter: 5 out of 5
[5 교차검증] Early stopping
1-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_1-1-alldata-phrase.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0168	 Train Acc:74.65 %  | 	Valid Loss:0.0145 	Valid Acc: 78.93 %

Validation loss decreased (inf --> 0.014513).  Saving model ...

[EPOCH:2]	 Train Loss:0.0130	 Train Acc:82.68 %  | 	Valid Loss:0.0134 	Valid Acc: 82.39 %

Validation loss decreased (0.014513 --> 0.013442).  Saving model ...

[EPOCH:3]	 Train Loss:0.0119	 Train Acc:85.62 %  | 	V


[EPOCH:2]	 Train Loss:0.0130	 Train Acc:82.01 %  | 	Valid Loss:0.0126 	Valid Acc: 83.33 %

Validation loss decreased (0.013242 --> 0.012575).  Saving model ...

[EPOCH:3]	 Train Loss:0.0113	 Train Acc:85.01 %  | 	Valid Loss:0.0136 	Valid Acc: 80.82 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0112	 Train Acc:85.12 %  | 	Valid Loss:0.0116 	Valid Acc: 84.28 %

Validation loss decreased (0.012575 --> 0.011627).  Saving model ...

[EPOCH:5]	 Train Loss:0.0106	 Train Acc:86.62 %  | 	Valid Loss:0.0133 	Valid Acc: 82.08 %

EarlyStopping counter: 1 out of 5

[EPOCH:6]	 Train Loss:0.0100	 Train Acc:87.25 %  | 	Valid Loss:0.0121 	Valid Acc: 83.33 %

EarlyStopping counter: 2 out of 5

[EPOCH:7]	 Train Loss:0.0103	 Train Acc:87.89 %  | 	Valid Loss:0.0117 	Valid Acc: 85.22 %

EarlyStopping counter: 3 out of 5

[EPOCH:8]	 Train Loss:0.0099	 Train Acc:86.68 %  | 	Valid Loss:0.0119 	Valid Acc: 85.22 %

EarlyStopping counter: 4 out of 5

[EPOCH:9]	 Train Loss:0.0091	 Train Acc:88.70 %


[EPOCH:6]	 Train Loss:0.0094	 Train Acc:87.77 %  | 	Valid Loss:0.0123 	Valid Acc: 83.02 %

EarlyStopping counter: 2 out of 5

[EPOCH:7]	 Train Loss:0.0088	 Train Acc:89.39 %  | 	Valid Loss:0.0108 	Valid Acc: 86.79 %

Validation loss decreased (0.011541 --> 0.010834).  Saving model ...

[EPOCH:8]	 Train Loss:0.0086	 Train Acc:88.70 %  | 	Valid Loss:0.0123 	Valid Acc: 83.65 %

EarlyStopping counter: 1 out of 5

[EPOCH:9]	 Train Loss:0.0078	 Train Acc:89.85 %  | 	Valid Loss:0.0111 	Valid Acc: 86.48 %

EarlyStopping counter: 2 out of 5

[EPOCH:10]	 Train Loss:0.0072	 Train Acc:90.37 %  | 	Valid Loss:0.0121 	Valid Acc: 83.96 %

EarlyStopping counter: 3 out of 5

[EPOCH:11]	 Train Loss:0.0073	 Train Acc:90.60 %  | 	Valid Loss:0.0116 	Valid Acc: 85.53 %

EarlyStopping counter: 4 out of 5

[EPOCH:12]	 Train Loss:0.0082	 Train Acc:88.75 %  | 	Valid Loss:0.0125 	Valid Acc: 83.96 %

EarlyStopping counter: 5 out of 5
[4 교차검증] Early stopping
../checkpoint/checkpoint_w2v_large_model60k_2-5-alldata-


[EPOCH:5]	 Train Loss:0.0105	 Train Acc:86.55 %  | 	Valid Loss:0.0123 	Valid Acc: 81.76 %

Validation loss decreased (0.012614 --> 0.012341).  Saving model ...

[EPOCH:6]	 Train Loss:0.0103	 Train Acc:87.18 %  | 	Valid Loss:0.0121 	Valid Acc: 83.65 %

Validation loss decreased (0.012341 --> 0.012122).  Saving model ...

[EPOCH:7]	 Train Loss:0.0095	 Train Acc:88.16 %  | 	Valid Loss:0.0123 	Valid Acc: 83.33 %

EarlyStopping counter: 1 out of 5

[EPOCH:8]	 Train Loss:0.0089	 Train Acc:88.45 %  | 	Valid Loss:0.0122 	Valid Acc: 82.39 %

EarlyStopping counter: 2 out of 5

[EPOCH:9]	 Train Loss:0.0088	 Train Acc:88.51 %  | 	Valid Loss:0.0130 	Valid Acc: 79.87 %

EarlyStopping counter: 3 out of 5

[EPOCH:10]	 Train Loss:0.0083	 Train Acc:89.20 %  | 	Valid Loss:0.0120 	Valid Acc: 81.45 %

Validation loss decreased (0.012122 --> 0.011999).  Saving model ...

[EPOCH:11]	 Train Loss:0.0088	 Train Acc:89.38 %  | 	Valid Loss:0.0124 	Valid Acc: 82.39 %

EarlyStopping counter: 1 out of 5

[EPOCH:12]

...


[EPOCH:7]	 Train Loss:0.0109	 Train Acc:86.33 %  | 	Valid Loss:0.0133 	Valid Acc: 83.96 %

Validation loss decreased (0.013791 --> 0.013297).  Saving model ...

[EPOCH:8]	 Train Loss:0.0102	 Train Acc:85.35 %  | 	Valid Loss:0.0131 	Valid Acc: 84.28 %

Validation loss decreased (0.013297 --> 0.013137).  Saving model ...

[EPOCH:9]	 Train Loss:0.0096	 Train Acc:86.79 %  | 	Valid Loss:0.0148 	Valid Acc: 79.87 %

EarlyStopping counter: 1 out of 5

[EPOCH:10]	 Train Loss:0.0100	 Train Acc:86.33 %  | 	Valid Loss:0.0138 	Valid Acc: 83.02 %

EarlyStopping counter: 2 out of 5

[EPOCH:11]	 Train Loss:0.0104	 Train Acc:85.64 %  | 	Valid Loss:0.0121 	Valid Acc: 84.91 %

Validation loss decreased (0.013137 --> 0.012094).  Saving model ...

[EPOCH:12]	 Train Loss:0.0099	 Train Acc:87.08 %  | 	Valid Loss:0.0131 	Valid Acc: 84.28 %

EarlyStopping counter: 1 out of 5

[EPOCH:13]	 Train Loss:0.0092	 Train Acc:88.18 %  | 	Valid Loss:0.0141 	Valid Acc: 83.33 %

EarlyStopping counter: 2 out of 5

[EPOCH:1

[4 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0173	 Train Acc:71.05 %  | 	Valid Loss:0.0141 	Valid Acc: 79.87 %

Validation loss decreased (inf --> 0.014078).  Saving model ...

[EPOCH:2]	 Train Loss:0.0129	 Train Acc:81.43 %  | 	Valid Loss:0.0141 	Valid Acc: 80.50 %

EarlyStopping counter: 1 out of 5

[EPOCH:3]	 Train Loss:0.0125	 Train Acc:83.10 %  | 	Valid Loss:0.0144 	Valid Acc: 79.56 %

EarlyStopping counter: 2 out of 5

[EPOCH:4]	 Train Loss:0.0120	 Train Acc:83.51 %  | 	Valid Loss:0.0141 	Valid Acc: 79.56 %

Validation loss decreased (0.014078 --> 0.014067).  Saving model ...

[EPOCH:5]	 Train Loss:0.0113	 Train Acc:84.89 %  | 	Valid Loss:0.0153 	Valid Acc: 77.99 %

EarlyStopping counter: 1 out of 5

[EPOCH:6]	 Train Loss:0.0107	 Train Acc:86.10 %  | 	Valid Loss:0.0138 	Valid Acc: 82.08 %

Validation loss decreased (0.014067 --> 0.013848).  Saving model ...

[EPOCH:7]	 Train Loss:0.0109	 Train Acc:85.81 %  | 	Valid Loss:0.0131 	Valid Acc: 83.33 %

Validation loss decreased (0.


[EPOCH:17]	 Train Loss:0.0198	 Train Acc:66.05 %  | 	Valid Loss:0.0176 	Valid Acc: 68.24 %

EarlyStopping counter: 3 out of 5

[EPOCH:18]	 Train Loss:0.0193	 Train Acc:68.24 %  | 	Valid Loss:0.0381 	Valid Acc: 39.94 %

EarlyStopping counter: 4 out of 5

[EPOCH:19]	 Train Loss:0.0188	 Train Acc:70.38 %  | 	Valid Loss:0.0520 	Valid Acc: 32.70 %

EarlyStopping counter: 5 out of 5
[3 교차검증] Early stopping
../checkpoint/checkpoint_w2v_large_model60k_21-4-alldata-phrase.pt
[4 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0235	 Train Acc:56.11 %  | 	Valid Loss:0.0228 	Valid Acc: 41.82 %

Validation loss decreased (inf --> 0.022820).  Saving model ...

[EPOCH:2]	 Train Loss:0.0227	 Train Acc:59.05 %  | 	Valid Loss:0.0211 	Valid Acc: 67.92 %

Validation loss decreased (0.022820 --> 0.021145).  Saving model ...

[EPOCH:3]	 Train Loss:0.0226	 Train Acc:59.28 %  | 	Valid Loss:0.0195 	Valid Acc: 66.04 %

Validation loss decreased (0.021145 --> 0.019505).  Saving model ...

[EPOCH:4]	 Train Loss:0.02


[EPOCH:5]	 Train Loss:0.0220	 Train Acc:55.02 %  | 	Valid Loss:0.0211 	Valid Acc: 60.38 %

EarlyStopping counter: 3 out of 5

[EPOCH:6]	 Train Loss:0.0219	 Train Acc:55.83 %  | 	Valid Loss:0.0218 	Valid Acc: 68.24 %

EarlyStopping counter: 4 out of 5

[EPOCH:7]	 Train Loss:0.0215	 Train Acc:56.52 %  | 	Valid Loss:0.0207 	Valid Acc: 69.18 %

Validation loss decreased (0.020807 --> 0.020674).  Saving model ...

[EPOCH:8]	 Train Loss:0.0216	 Train Acc:56.99 %  | 	Valid Loss:0.0254 	Valid Acc: 40.57 %

EarlyStopping counter: 1 out of 5

[EPOCH:9]	 Train Loss:0.0214	 Train Acc:57.91 %  | 	Valid Loss:0.0215 	Valid Acc: 68.24 %

EarlyStopping counter: 2 out of 5

[EPOCH:10]	 Train Loss:0.0209	 Train Acc:62.07 %  | 	Valid Loss:0.0241 	Valid Acc: 68.24 %

EarlyStopping counter: 3 out of 5

[EPOCH:11]	 Train Loss:0.0209	 Train Acc:59.12 %  | 	Valid Loss:0.0182 	Valid Acc: 68.24 %

Validation loss decreased (0.020674 --> 0.018228).  Saving model ...

[EPOCH:12]	 Train Loss:0.0209	 Train Acc:61.6


[EPOCH:3]	 Train Loss:0.0225	 Train Acc:54.62 %  | 	Valid Loss:0.0232 	Valid Acc: 68.24 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0222	 Train Acc:56.12 %  | 	Valid Loss:0.0357 	Valid Acc: 31.76 %

EarlyStopping counter: 2 out of 5

[EPOCH:5]	 Train Loss:0.0217	 Train Acc:56.47 %  | 	Valid Loss:0.0248 	Valid Acc: 68.24 %

EarlyStopping counter: 3 out of 5

[EPOCH:6]	 Train Loss:0.0217	 Train Acc:56.76 %  | 	Valid Loss:0.0198 	Valid Acc: 68.24 %

EarlyStopping counter: 4 out of 5

[EPOCH:7]	 Train Loss:0.0217	 Train Acc:57.39 %  | 	Valid Loss:0.0256 	Valid Acc: 68.24 %

EarlyStopping counter: 5 out of 5
[3 교차검증] Early stopping
../checkpoint/checkpoint_w2v_large_model60k_23-4-alldata-phrase.pt
[4 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0231	 Train Acc:54.56 %  | 	Valid Loss:0.0316 	Valid Acc: 37.11 %

Validation loss decreased (inf --> 0.031598).  Saving model ...

[EPOCH:2]	 Train Loss:0.0228	 Train Acc:55.71 %  | 	Valid Loss:0.0258 	Valid Acc: 67.92 %

Validati

# test

In [26]:
for num, valid_fold in enumerate(all_valid_accs):
    print("-----\n")
    avg_train = []
    avg_valid = []
    for num_fold,valid in enumerate(valid_fold):
        print("{}번 모델. {} 폴드.  train :{:.4f}/ valid: {:.4f}".format(num+1,num_fold+1,all_train_accs[num][num_fold],valid) )
        avg_train.append(all_train_accs[num][num_fold])
        avg_valid.append(valid)
    print("* 평균 train acc : {:.4f}% / 평균 valid acc : {:.4f}% ".format(np.array(avg_train).mean(), np.array(avg_valid).mean()) )

-----

1번 모델. 1 폴드.  train :86.9515/ valid: 81.4465
1번 모델. 2 폴드.  train :85.2771/ valid: 86.4780
1번 모델. 3 폴드.  train :82.3326/ valid: 81.4465
1번 모델. 4 폴드.  train :87.7163/ valid: 83.9623
1번 모델. 5 폴드.  train :83.7370/ valid: 86.1635
* 평균 train acc : 85.2029% / 평균 valid acc : 83.8994% 
-----

2번 모델. 1 폴드.  train :88.8568/ valid: 84.9057
2번 모델. 2 폴드.  train :86.4896/ valid: 88.0503
2번 모델. 3 폴드.  train :87.7021/ valid: 81.4465
2번 모델. 4 폴드.  train :85.8708/ valid: 85.5346
2번 모델. 5 폴드.  train :85.1211/ valid: 84.2767
* 평균 train acc : 86.8081% / 평균 valid acc : 84.8428% 
-----

3번 모델. 1 폴드.  train :89.3764/ valid: 83.3333
3번 모델. 2 폴드.  train :90.6467/ valid: 87.1069
3번 모델. 3 폴드.  train :87.7021/ valid: 83.0189
3번 모델. 4 폴드.  train :89.3887/ valid: 86.7925
3번 모델. 5 폴드.  train :90.0231/ valid: 85.8491
* 평균 train acc : 89.4274% / 평균 valid acc : 85.2201% 
-----

4번 모델. 1 폴드.  train :90.1848/ valid: 84.5912
4번 모델. 2 폴드.  train :87.1824/ valid: 86.1635
4번 모델. 3 폴드.  train :89.2032/ valid: 81.4465
4번 

In [22]:
for num,valid in enumerate(valid_accs):
    print("{}번 모델. train :{:.4f}/ valid: {:.4f}".format(num+1,train_accs[num],valid) )

1번 모델. train :84.4037/ valid: 87.7133
2번 모델. train :84.7313/ valid: 87.3720
3번 모델. train :86.1730/ valid: 88.3959
4번 모델. train :88.7287/ valid: 87.0307
5번 모델. train :82.9620/ valid: 83.2765
6번 모델. train :86.4351/ valid: 86.3481
7번 모델. train :86.6317/ valid: 87.3720
8번 모델. train :86.3041/ valid: 86.6894
9번 모델. train :86.8283/ valid: 87.0307
10번 모델. train :86.5662/ valid: 86.6894
11번 모델. train :86.3041/ valid: 87.7133
12번 모델. train :87.5491/ valid: 86.0068
13번 모델. train :86.6972/ valid: 83.6177
14번 모델. train :85.8453/ valid: 81.9113
15번 모델. train :81.1271/ valid: 86.0068
16번 모델. train :83.2241/ valid: 84.3003
17번 모델. train :84.3381/ valid: 85.6655
18번 모델. train :85.8453/ valid: 84.3003
19번 모델. train :81.1271/ valid: 83.9590
20번 모델. train :84.0105/ valid: 84.6416
21번 모델. train :83.8794/ valid: 86.0068
22번 모델. train :53.3421/ valid: 65.1877
23번 모델. train :53.9974/ valid: 65.5290
24번 모델. train :55.4391/ valid: 60.4096


In [23]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_transformer_all.csv")

In [ ]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_encoder.csv")

In [ ]:
#encoder layer
test_grid()

In [27]:
# 24 레이어
test_grid()

1번 모델 / 1번 fold
Accuracy : 82.1608% 
Precision (pathology 예측한 것중 맞는 것) : 0.9032
recall (실제 pathology 중  예측이 맞는 것) : 0.8266
f score : 0.8034 
[[224  47]
 [ 24 103]]
-----
1번 모델 / 2번 fold
Accuracy : 81.6583% 
Precision (pathology 예측한 것중 맞는 것) : 0.8867
recall (실제 pathology 중  예측이 맞는 것) : 0.8376
f score : 0.7951 
[[227  44]
 [ 29  98]]
-----
1번 모델 / 3번 fold
Accuracy : 83.6683% 
Precision (pathology 예측한 것중 맞는 것) : 0.9120
recall (실제 pathology 중  예측이 맞는 것) : 0.8413
f score : 0.8194 
[[228  43]
 [ 22 105]]
-----
1번 모델 / 4번 fold
Accuracy : 83.1658% 
Precision (pathology 예측한 것중 맞는 것) : 0.8835
recall (실제 pathology 중  예측이 맞는 것) : 0.8672
f score : 0.8083 
[[235  36]
 [ 31  96]]
-----
1번 모델 / 5번 fold
Accuracy : 82.4121% 
Precision (pathology 예측한 것중 맞는 것) : 0.8851
recall (실제 pathology 중  예측이 맞는 것) : 0.8524
f score : 0.8016 
[[231  40]
 [ 30  97]]
-----
----
* 1번 모델 82.6131% / score: 0.8056
----
2번 모델 / 1번 fold
Accuracy : 83.1658% 
Precision (pathology 예측한 것중 맞는 것) : 0.8923
recall (실제 pathology 중  예측이

10번 모델 / 2번 fold
Accuracy : 81.1558% 
Precision (pathology 예측한 것중 맞는 것) : 0.9016
recall (실제 pathology 중  예측이 맞는 것) : 0.8118
f score : 0.7937 
[[220  51]
 [ 24 103]]
-----
10번 모델 / 3번 fold
Accuracy : 78.8945% 
Precision (pathology 예측한 것중 맞는 것) : 0.9156
recall (실제 pathology 중  예측이 맞는 것) : 0.7601
f score : 0.7753 
[[206  65]
 [ 19 108]]
-----
10번 모델 / 4번 fold
Accuracy : 80.9045% 
Precision (pathology 예측한 것중 맞는 것) : 0.9149
recall (실제 pathology 중  예측이 맞는 것) : 0.7934
f score : 0.7939 
[[215  56]
 [ 20 107]]
-----
10번 모델 / 5번 fold
Accuracy : 80.6533% 
Precision (pathology 예측한 것중 맞는 것) : 0.9181
recall (실제 pathology 중  예측이 맞는 것) : 0.7860
f score : 0.7921 
[[213  58]
 [ 19 108]]
-----
----
* 10번 모델 80.4523% / score: 0.7890
----
11번 모델 / 1번 fold
Accuracy : 79.8995% 
Precision (pathology 예측한 것중 맞는 것) : 0.9170
recall (실제 pathology 중  예측이 맞는 것) : 0.7749
f score : 0.7849 
[[210  61]
 [ 19 108]]
-----
11번 모델 / 2번 fold
Accuracy : 82.4121% 
Precision (pathology 예측한 것중 맞는 것) : 0.9170
recall (실제 pathology

19번 모델 / 3번 fold
Accuracy : 77.8894% 
Precision (pathology 예측한 것중 맞는 것) : 0.9336
recall (실제 pathology 중  예측이 맞는 것) : 0.7269
f score : 0.7686 
[[197  74]
 [ 14 113]]
-----
19번 모델 / 4번 fold
Accuracy : 81.1558% 
Precision (pathology 예측한 것중 맞는 것) : 0.9118
recall (실제 pathology 중  예측이 맞는 것) : 0.8007
f score : 0.7957 
[[217  54]
 [ 21 106]]
-----
19번 모델 / 5번 fold
Accuracy : 80.6533% 
Precision (pathology 예측한 것중 맞는 것) : 0.9042
recall (실제 pathology 중  예측이 맞는 것) : 0.8007
f score : 0.7896 
[[217  54]
 [ 23 104]]
-----
----
* 19번 모델 79.2965% / score: 0.7795
----
20번 모델 / 1번 fold
Accuracy : 80.1508% 
Precision (pathology 예측한 것중 맞는 것) : 0.9248
recall (실제 pathology 중  예측이 맞는 것) : 0.7712
f score : 0.7884 
[[209  62]
 [ 17 110]]
-----
20번 모델 / 2번 fold
Accuracy : 79.8995% 
Precision (pathology 예측한 것중 맞는 것) : 0.9207
recall (실제 pathology 중  예측이 맞는 것) : 0.7712
f score : 0.7855 
[[209  62]
 [ 18 109]]
-----
20번 모델 / 3번 fold
Accuracy : 80.6533% 
Precision (pathology 예측한 것중 맞는 것) : 0.9330
recall (실제 pathology

In [ ]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_'+str(data_ind)+'.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))


